In [ ]:
import shutil
from pathlib import Path

transformers_path = Path("/opt/conda/lib/python3.7/site-packages/transformers")

input_dir = Path("../input/deberta-v2-3-fast-tokenizer")

convert_file = input_dir / "convert_slow_tokenizer.py"
conversion_path = transformers_path/convert_file.name

if conversion_path.exists():
    conversion_path.unlink()

shutil.copy(convert_file, transformers_path)
deberta_v2_path = transformers_path / "models" / "deberta_v2"

for filename in ['tokenization_deberta_v2.py', 'tokenization_deberta_v2_fast.py', "deberta__init__.py"]:
    if str(filename).startswith("deberta"):
        filepath = deberta_v2_path/str(filename).replace("deberta", "")
    else:
        filepath = deberta_v2_path/filename
    if filepath.exists():
        filepath.unlink()

    shutil.copy(input_dir/filename, filepath)

In [ ]:
import torch
from transformers import DistilBertForQuestionAnswering, DistilBertTokenizerFast
model = torch.load("../input/finetunedqadistilbert/FINE_TUNED_DISTILBERT_QA_V0")
tokenizer = DistilBertTokenizerFast.from_pretrained("../input/distilbertbaseuncased/")

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import ast
from tqdm import tqdm
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer, AutoConfig

In [ ]:
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

# os.system('pip uninstall -y transformers')
# os.system('python -m pip install --no-index --find-links=../input/nbme-pip-wheels transformers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# move model over to detected device
model.to(device)

In [ ]:
test = pd.read_csv('../input/nbme-score-clinical-patient-notes/test.csv')
test.head()

In [ ]:
features = pd.read_csv('../input/nbme-score-clinical-patient-notes/features.csv')
def preprocess_features(features):
    features.loc[27, 'feature_text'] = "Last-Pap-smear-1-year-ago"
    return features
features = preprocess_features(features)
patient_notes = pd.read_csv('../input/nbme-score-clinical-patient-notes/patient_notes.csv')

print(f"features.shape: {features.shape}")
display(features.head())
print(f"patient_notes.shape: {patient_notes.shape}")
display(patient_notes.head())

In [ ]:
test = test.merge(features, on=['feature_num', 'case_num'], how='left')
test = test.merge(patient_notes, on=['pn_num', 'case_num'], how='left')
display(test.head())

In [ ]:
test_contexts = test['pn_history'].copy().tolist()
test_questions = test['feature_text'].copy().tolist()

In [ ]:
test_encodings = tokenizer(test_contexts, test_questions, truncation=True, padding=True)

In [ ]:
#mediumdan aldığımız kodda bu vardı
#https://towardsdatascience.com/how-to-fine-tune-a-q-a-transformer-86f91ec92997
class TrainDatasetV2(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
test_dataset = TrainDatasetV2(test_encodings)

In [ ]:
model.eval()
# initialize validation set data loader
test_loader = DataLoader(test_dataset, batch_size=4)
# initialize list to store accuracies
preds_start = []
preds_end = []
# loop through batches
for batch in test_loader:
    # we don't need to calculate gradients as we're not training
    with torch.no_grad():
        # pull batched items from loader
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        # make predictions
        outputs = model(input_ids, attention_mask=attention_mask)
        # pull prediction tensors out and argmax to get predicted tokens
        start_pred = torch.argmax(outputs['start_logits'], dim=1)
        end_pred = torch.argmax(outputs['end_logits'], dim=1)
        
        start_np = start_pred.detach().cpu().numpy()
        end_np = end_pred.detach().cpu().numpy()
        
        preds_start.extend(start_np)
        preds_end.extend(end_np)

In [ ]:
preds_all_sub = []

for i in range(0, len(preds_start)):
    temp = []
    temp.append(preds_start[i])
    temp.append(preds_end[i])
    preds_all_sub.append(temp)

In [ ]:
def get_results(char_probs, th=0.5):
    results = []
    for char_prob in char_probs:
        result = np.where(char_prob >= th)[0] + 1
        result = [list(g) for _, g in itertools.groupby(result, key=lambda n, c=itertools.count(): n - next(c))]
        result = [f"{min(r)} {max(r)}" for r in result]
        result = ";".join(result)
        results.append(result)
        
    return results

In [ ]:
def get_char_probs(texts, predictions, tokenizer):
    results = [np.zeros(len(t)) for t in texts]
    for i, (text, prediction) in enumerate(zip(texts, predictions)):
        encoded = tokenizer(text, 
                            add_special_tokens=True,
                            return_offsets_mapping=True)
        for idx, (offset_mapping, pred) in enumerate(zip(encoded['offset_mapping'], prediction)):
            start = offset_mapping[0]
            end = offset_mapping[1]
            results[i][start:end] = pred
            
    return results

char_probs = get_char_probs(test['pn_history'].values, preds_start, tokenizer)
predictions.append(char_probs)

In [ ]:
preds_all_sub

In [ ]:
results = []
for result in preds_all_sub: 
    result = [f"{min(result)} {max(result)}"]
    result = ";".join(result)
    results.append(result)

In [ ]:
results

In [ ]:
test['location'] = results
display(test.head())
test[['id', 'location']].to_csv('submission.csv', index=False)

In [ ]:
"""
test['start'] = preds_start
test['end'] = preds_end

test['start'] = test['start'].astype("string")
test['end'] = test['end'].astype("string")
"""

In [ ]:
#test['location'] = test['start'] +" "+test['end']

test['start'] = preds_start
test['end'] = preds_end

results=f"{test['start']} {test['end']}"

test['location'] = test['start'].map(str) + " "+ test['end'].map(str)

test = test.replace(" ", np.nan)

test['location'] = preds_all_sub

test.head()

test[['id', 'location']].to_csv('submission.csv', index=False)

test.head()

#modeli private kaydedelim yeni train ve inference notebookları açalım 
#discussion ve code kısmını okuyalım 
#deberta base 